In [4]:
import time
from braket.aws import AwsSession
from braket.aws import AwsQuantumJob
from braket.jobs.local import LocalQuantumJob
from braket.jobs import hybrid_job

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

In [ ]:
import sys
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
import json
module_path = os.path.abspath(os.path.join('/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control'))
if module_path not in sys.path:
    sys.path.append(module_path)

from quantumenvironment import QuantumEnvironment
from template_configurations import gate_q_env_config
from helper_functions import load_agent_from_yaml_file
from ppo import make_train_ppo

import logging
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s INFO %(message)s", # hardcoded INFO level
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,
)

In [1]:
from braket.aws import AwsDevice
from braket.jobs import save_job_result
from braket.jobs.metrics import log_metric
from braket.tracking import Tracker

from qiskit_braket_provider import BraketLocalBackend, AWSBraketProvider, AWSBraketBackend, AWSBraketJob

In [2]:
def calibrate_gate(num_total_updates):
    print("Job started!!!!!")

    from quantumenvironment import QuantumEnvironment
    from template_configurations import gate_q_env_config
    from helper_functions import load_agent_from_yaml_file
    from ppo import make_train_ppo

    braket_task_costs = Tracker().start()
    provider = AWSBraketProvider()
    backend = provider.get_backend('SV1')
    q_env = QuantumEnvironment(gate_q_env_config)
    q_env.backend = backend
    ppo_params, network_params  = load_agent_from_yaml_file(file_path='/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/template_configurations/agent_config.yaml')
    agent_config = {**ppo_params, **network_params}
    ppo_agent = make_train_ppo(agent_config, q_env)

    # training_results = ppo_agent(total_updates=hyperparams["num_updates"], print_debug=True, num_prints=40)
    training_results = ppo_agent(total_updates=num_total_updates, print_debug=False, num_prints=40)
    # save_job_result({ "measurement_counts": training_results })

    print("Job completed!!!!!")

    training_results['task_summary'] = braket_task_costs.quantum_tasks_statistics()
    training_results['estimated cost'] = float(
            braket_task_costs.qpu_tasks_cost() + braket_task_costs.simulator_tasks_cost()
        )

    return training_results

In [5]:
# required_files = [
#     '/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/quantumenvironment.py',
#     '/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/ppo.py',
#     '/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/helper_functions.py',
#     '/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/template_configurations/qiskit/q_env_config.py',
# ]
required_files = '00_needed_files'

@hybrid_job(device=None, include_modules=required_files)
def calibrate_gate_hybrid_job(num_total_updates):
    return calibrate_gate(num_total_updates)

In [6]:
job = calibrate_gate_hybrid_job(num_total_updates=10)

In [10]:
job.state()

'FAILED'

In [ ]:
%%time
job.result()